# Basic Multi-Agent Example with AutoGen

This notebook implements a simple multi-agent system using AutoGen. 

In [11]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
import sys

sys.path.append(os.path.abspath(".."))
from utilities import (
    create_chat_completion_client,    
    adjust_properties_for_fixed_response,    
    load_model_config,
)

In [12]:
model = load_model_config()
# model = load_model_config(model_name="gpt-oss:20b")
# model = load_model_config(model_name="qwen3")

Using model: gpt-4.1-nano type: openai


Set the joke's topic
----------------------

In [13]:
topic = "Animals"

Select Model and Agent Configuration
-----------------------------

In [14]:
agentConfig = {
    "Author": dict(
        system_message="Write a SHORT joke above the provided topic."
    ),
    "SmartassEditor": dict(
        system_message="Analyze the joke, explain why it is funny or not funny, and make it funnier without making it longer.",
    ),
}

# For testing, we can fix an agent's response to a known answer
#adjust_properties_for_fixed_response(agentConfig["Author"], "What do you call a magic dog? A Labracadabrador.")

In [15]:
# Now create model client
model_client = create_chat_completion_client(model)

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    agents.append(
        AssistantAgent(
            name=agent_type,
            **props,  # type: ignore
            model_client=model_client,
        )
    )

In [16]:
# Set up termination conditions
termination = MaxMessageTermination(max_messages=10) #len(agents))

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)


Start the System
===================

In [17]:
async def invent_joke(topic: str):
    start_time_s = timeit.default_timer()
    result = await team.run(task=f"Invent a short joke about: {topic}")
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 Generation result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await invent_joke(topic)

🔍 Generation result (took 1.23 sec.) resulted in 3 messages.


Explore Results
===============

In [18]:
from autogen_agentchat.messages import BaseTextChatMessage

print("Stop reason:", result.stop_reason)
assert isinstance(result.messages[-1], BaseTextChatMessage)

print("Message Sequence:")
for m in result.messages:
    assert isinstance(m, BaseTextChatMessage), f"Unexpected message type: {type(m)}"
    message = m.content
    print(f" - {m.source}: {message}")


Stop reason: Maximum number of turns 2 reached.
Message Sequence:
 - user: Invent a short joke about: Animals
 - Author: Why did the scarecrow become friends with the cow? Because he knew how to moo-ve people!
 - SmartassEditor: This joke is funny because it plays on the word “moo,” which is the sound cows make, combining it with “move” to create a pun. The humor comes from the unexpected link between a scarecrow, a farm figure, and a cow, highlighting the playful wordplay.

To make it funnier without making it longer:
**"Why did the scarecrow become friends with the cow? Because he was outstanding in his field—and knew how to moo-ve!"**
